#### Общие правила
- Сделанная домашняя работа - jupyter-тетрадка.
- Пожалуйста, присваивайте блокам кода номера и/или заголовки, соответствующие заданиям. Код должен содержать комментарии, ход мысли и решения должен быть понятен из вашей тетрадки
- Максимальный балл за домашнюю работу - 20. Вы не обязаны выполнять все пункты, чтобы набрать максимальное количество баллов
- Можно использовать чужой код, но обязательно ставьте ссылки на источник
- **Deadline** - 25.04.2019

## AGRR: Automatic Gapping Resolution for Russian

#### Краткий ликбез
Примеры гэппинга в русском
 - _Ей он рассказывает одно, а нам — совершенно другое_  
 - _Кто любит арбуз, а кто — свиной хрящик_  
 - _Дайте мне две пятерки, а я вам десятку_
 - _Тогда я принял ее за итальянку, а его за шведа_
 

train + dev включают в себя больше 20 тысяч примеров, примерно треть из них иллюстрирует явление гэппинга в русском.  <br>
Гэппинг - явление, при котором в одной из сочиненных (иногда подчиненных) клауз, опускается повторяющийся предикат, но остаются выраженными его участники.
 - _Тогда я принял ее за итальянку, а его ~~принял~~ за шведа_


Терминология для описания элементов гэппинга разнится, мы используем следующую:

При разметке предложения мы выделяем следующие элементы (терминология разнится в разных источниках, бояться не нужно):
- ремнанты **R1** и **R2** - участники эллиптированного предиката (в нашем примере _его_ и _за шведа_)
- корреляты ремнантов  **cR1** и **cR2** - соответсвующие группы в полной клаузе (в нашем примере _ее_ и _за итальянку_)
- позиция опущенного (эллиптированного) предиката  **V**
- вершина конролирующего предиката **cV** (в нашем примере _принял_ из первой клаузы)

Посмотрите на пример со скобочной разметкой:

 - _Тогда я  **cV[** принял **cV]**  **cR1[** ее **cR1]**  **cR2[** за итальянку **cR2]**, а  **R1[** его **R1]**   **V[]**  **cR2[** за шведа **cR2]**._

Есть много теоретических работ, посвященных этому виду эллипсиса (прикладных решений практически нет). Желающим: 

- Лингвистическая сторона дела:
[Ross 1970](https://babel.ucsc.edu/~hank/Ross_Gapping.pdf) 
[Jackendoff 1971](https://babel.ucsc.edu/~hank/jackendoff_related_rules.pdf) 
[Steedman 1989](https://repository.upenn.edu/cgi/viewcontent.cgi?article=1861&context=cis_reports) 
[Coppock 2001](https://www.linguistics.northwestern.edu/documents/award-winners/linguistics-undergraduate-award-past-winner-coppock.pdf)
[Merchant 2016](http://home.uchicago.edu/merchant/pubs/ellipsis.revised.pdf) 

- Подходы к решениям:
[Bogdanov  2012](http://www.dialog-21.ru/media/1316/89.pdf) 
[Droganova 2018](http://www.aclweb.org/anthology/W18-6006) 
[Schuster 2018](https://arxiv.org/pdf/1804.06922.pdf) 



**NB**: Данные собраны, размечены и подготовлены к обучению, так что если вас пугает, что вы не очень понимаете явление, с которым придется работать, это не так критично, не пугайтесь. <br>
Задачу можно воспринимать как классификацию последовательности, в этом смысле она не отличается от задачи NER или POS-tagging. 

 
### Формат данных
Скобочная запись удобна для оценки разметки глазами. Сама разметка хранится в следующем виде:
csv файл, колонки разделены табуляцией.
Первая колонка - сам текст. Вторая колонка - 0 или 1 (отсутсвие или наличие гэппинга в предложении).
Следующие 6 колонок содержат оффсеты подстрок, соответсвующих элементам гэппинга(**cV, cR1, cR2, V, R1, R2**). Ячейка может быть пустой (не все элементы обязательны), может содержать как одну пару оффсетов (разделены через двоеточие), так и несколько пар (разделены через пробел, порядок пар не важен).
Посмотрите на пример:
 
**Input**  
>Аналогичным образом, среднегодовой прирост ВВП на душу населения, который в странах, расположенных к югу от Сахары, составлял в период с 1965 по 1973 год 3 процента, упал с 1980 до 1986 года на 2,8 процента, в 1987 году - на 4,4 процента и в 1989 году - на 0,5 процента.

**Output**  

| class | cV  | cR1  | cR2 | V | R1  | R2  |
| :---  | :---  | :---  | :---  | :---  | :---  | :---  |
| 1| 166:170 | 171:190 | 191:206  | 222:222 254:254  | 208:219 240:251 | 222:237 254:269 |


Такая разметка соответствует следующей скобочной записи:

 - _Аналогичным образом, среднегодовой прирост ВВП на душу населения, который в странах, расположенных к югу от Сахары, составлял в период с 1965 по 1973 год 3 процента,  **cV[** упал **cV]**  **cR1[** с 1980 до 1986 года **cR1]**  **cR2[** на 2,8 процента **cR2]**,   **R1[** в 1987 году **R1]** — **V[]** **R2[** на 4,4 процента **R2]** и  **R1[** в 1989 году **R1]** —  **V[]** **R2[** на 0,5 процента **R2]**._

## Описание заданий

#### 1. Подготовка данных (2 балла)
Скачайте данные (train, dev, test). Для работы с таблицей используйте pandas. 
Используйте параметр quoting - иначе pandas вольно обращается с кавычками в тексте и оффсеты могут съехать
> import pandas as pd <br>
import csv <br>
> train = pd.read_csv('train.csv',sep = '\t', quoting=csv.QUOTE_NONE) 

Посчитайте, сколько в каждом из файлов примеров из класса 0 и из класса 1.


In [1]:
import pandas as pd 
import csv 
train = pd.read_csv('train.csv',sep = '\t', quoting=csv.QUOTE_NONE)
test = pd.read_csv('test.csv',sep = '\t', quoting=csv.QUOTE_NONE)
dev = pd.read_csv('dev.csv',sep = '\t', quoting=csv.QUOTE_NONE)

In [14]:
train["class"].value_counts().to_frame()

,class
0,10864
1,5542


In [15]:
test["class"].value_counts().to_frame()

,class
0,1365
1,680


In [16]:
dev["class"].value_counts().to_frame()

,class
0,2760
1,1382


#### 2. Анализ разметки (2 балла)
Стандартный пример содержит по паре оффсетов для каждой метки **cV, cR1, cR2, V, R1, R2** , но есть и другие варианты. Посчитайте сколько их и какие случаи им соответствуют.
Представьте результаты в понятном виде, например в таблице.

In [129]:
# 0 вариантов, когда cV == NaN (train)
train[train["class"] == 1][train["cV"].isna()]

/Users/isupovand/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,text,class,cV,cR1,cR2,V,R1,R2


In [59]:
# 1 вариант, когда cR1 == NaN (train)
train[train["class"] == 1][train["cR1"].isna()]

/Users/isupovand/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,text,class,cV,cR1,cR2,V,R1,R2
8104,Вот поэтому с нашей стороны пытаться изобразит...,1,37:47,NaN,48:76,89:89,80:86,89:107


In [55]:
train["text"][8104]

'Вот поэтому с нашей стороны пытаться изобразить Господа как абсолютное добро, а Сатану – как абсолютное зло,\xa0– банально и глупо.'

In [133]:
# 77 вариантов, когда cR2 и R2 == NaN (train)
train[train["class"] == 1][train["cR2"].isna() & train["R2"].isna()]

/Users/isupovand/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,text,class,cV,cR1,cR2,V,R1,R2
14,"Но Христианин сказал Упрямому: — Нет, сосед, л...",1,105:112,113:139,NaN,156:156,156:169,NaN
36,Неправильное или слишком быстрое разогревание ...,1,52:57,67:75,NaN,105:105,105:145,NaN
95,И мы прогнозируем увеличение товарооборота на ...,1,5:17,43:58,NaN,97:97,97:112,NaN
124,Списание дебиторской задолженности по финансов...,1,58:63,74:135,NaN,161:161,161:207,NaN
142,"Судья посмотрела на свои часы, затем – на меня.",1,6:16,17:29,NaN,39:39,39:46,NaN
411,В лучшем случае этот обычай представляется не ...,1,28:42,43:98,NaN,113:113,113:148,NaN
598,Круг в основном имеет голубоватый цвет и лишь ...,1,16:21,22:33,NaN,65:65,65:76,NaN
617,"Кто любит лето, кто весну,",1,4:9,10:14,NaN,20:20,20:25,NaN
852,"Переступив через порог, я обнаружил небольшую ...",1,26:35,36:75,NaN,89:89,89:128,NaN
1168,Судебное следствие начинается с изложения госу...,1,19:29,30:106,NaN,150:150,150:191,NaN


In [63]:
# пример
train["text"][95]

'И мы прогнозируем увеличение товарооборота на 70 процентов, а по информации российской стороны – на 40 процентов.'

In [65]:
# 0 вариантов, когда R1 == NaN (train)
train[train["class"] == 1][train["R1"].isna()]

/Users/isupovand/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,text,class,cV,cR1,cR2,V,R1,R2


In [74]:
import re

In [139]:
# все варианты, где в столбце V больше 1 слайса (в тех же примерах такое же количество слайсов в R1 и R2)
train[train["class"] == 1][train[train["class"] == 1]["V"].apply(lambda x: True if re.findall(" ", x) else False)]

,text,class,cV,cR1,cR2,V,R1,R2
117,Индекс промышленного производства за январь-фе...,1,60:68,7:33,69:75,109:109 147:147,77:106 118:144,109:115 147:153
169,Больше выпущено товарной пищевой рыбной продук...,1,7:15,16:49,71:78,101:101 118:118,80:98 111:115,101:109 118:128
174,"Я смотрел, и каждая улочка казалась мне изящно...",1,27:35,13:26,40:54,73:73 103:103,56:70 84:100,73:80 103:115
211,В феврале на предприятиях металлургического ко...,1,120:131,113:119,132:141,151:151 172:172 191:191,143:148 162:169 183:188,151:160 172:181 191:200
292,"Сначала идет недовольство собой, потом миром, ...",1,8:12,0:7,26:31,39:39 52:52,33:38 46:51,39:44 52:75
539,"Кому то нравится Дженифер Лопес, кому то Мадон...",1,8:16,0:7,17:31,41:41 58:58,33:40 50:57,41:48 58:73
688,Выбросы загрязняющих веществ по электростанция...,1,193:202,177:192,203:211,229:229 255:255,213:226 239:252,229:237 255:262
755,"Например, в Польше за период 1999–2004 гг. про...",1,84:92,43:83,93:109,144:144 192:192,111:141 154:189,144:150 192:216
786,"Приветливость сменяется агрессивностью, слезы ...",1,14:23,0:13,24:38,48:48 66:66,40:45 56:63,48:54 66:76
792,"Кто-то любит коллекционировать марки, кто-то л...",1,7:12,0:6,13:36,45:45 83:83,38:44 76:82,45:74 83:95


In [158]:
# запишем значения нужных индексов и сами слайсы в виде списков в специальные переменные
indexesV = list(train[train["class"] == 1][train[train["class"] == 1]["V"].apply(lambda x: True if re.findall(" ", x) else False)].index.values)
longV = train["V"][train["class"] == 1][train[train["class"] == 1]["V"].apply(lambda x: True if re.findall(" ", x) else False)].to_list()

In [166]:
from collections import Counter, defaultdict
longV_count = Counter()
count = 0
longV_indexes = defaultdict(list)
i = 0
j = 0
while i < len(longV) and j < len(indexesV):
    longV_count[len(longV[i].split())] += 1
    longV_indexes[len(longV[i].split())].append(indexesV[j])
    i += 1
    j += 1

In [192]:
for i in longV_count.keys():
    print("В", longV_count[i], "предложениях по", i, "слайсов, вот пример:", train["text"][longV_indexes[i][0]])

В 315 предложениях по 2 слайсов, вот пример: Индекс промышленного производства за январь-февраль 2008 г. составил 106,0%, инвестиций в основной капитал - 120,2% и оборота розничной торговли - 116,3%.
В 45 предложениях по 3 слайсов, вот пример: В феврале на предприятиях металлургического комплекса области по сравнению с январем среднесуточное производство чугуна уменьшилось на 23,5 %, стали — на 22,9 %, проката - на 28,2 %, кокса — на 23,7 %.
В 7 предложениях по 4 слайсов, вот пример: Индекс Xetra DAX-30 (Германия) повысился на 4,9%, индекс CAC-40 (Франция) — на 3,7%, индекс FTSE-100 (Великобритания) — на 1,6%, индекс Nikkei 225 (Япония) — на 8,9%, индекс Hang Seng (Гонконг) — на 2,1%.
В 2 предложениях по 5 слайсов, вот пример: Домохозяйки становятся телеведущими, телеведущие - писателями, писатели - музыкантами, музыканты - актерами, актеры - политиками, а гастарбайтеры - мастерами суши.


#### 3. Конвертер разметки-1 (1 балл)
Напишите функцию, переводящую разметку по оффсетам в IOB/IO/BIOS(на выбор) формат и обратно. Она пригодится вам при формировании входа для сети для решения классификации последовательности.

Напоминание про IOB(ES) разметку:
Каждому элементу последовательности (например токену или символу) сопоставляется одна из типов меток: O - outside, B - begin,
I - inside, E - end, s - single, в зависимости от того, входит ли этот элемент в интересующую нас подстроку. Например, в задаче NER метка B-LOC означает, что данный элемент - первый в подстроке, соответствующей названию локации. 

Пример подходящей для нашей задачи IOB-разметки (на уровне токенов):

| Тогда | я   | принял  | ее    | за    | итальянку  | а   | его  | за   | шведа |
| :---  | :---| :---    | :---  | :---  | :---       | :---| :--- | :--- | :---  |
| O     | O   | B-cV    | B-cR1 | B-cR2 | I-cR2      | O   | B-R1 | B-R2 | I-R2  |


In [307]:
def parse_iob_tags(index):
    sent = train["text"][index]
    words = defaultdict()
    tagged_words = set()
    tags = ["cV", "cR1", "cR2", "R1", "R2"]
    for tag in tags:
        # парсим слайсы из датасета
        slices = []
        for element in train[tag][index].split():
            nums = element.split(":")
            for num in nums:
                slices.append(int(num))
        start = []
        end = []
        for n in range(len(slices)):
            if n % 2 == 0:
                start.append(slices[n])
            else:
                end.append(slices[n])
        # ставим метки словам согласно слайсам        
        i = 0
        j = 0
        while i < len(start) and j < len(end):
            word = sent[start[i]:end[j]]
            word_list = word.split()
            for i, w in enumerate(word_list):
                if i == 0:
                    words[w] = "B-" + tag
                else:
                    words[w] = "I-" + tag
                tagged_words.add(w)
            i += 1
            j += 1
        # ищем слова, которые не вошли в уже размеченные, им присваиваем метку О
        all_words = set(sent[:-1].split())
        outside_words = all_words - tagged_words
        for w in outside_words:
            words[w] = "O"
    return words

In [308]:
def iob_tagging(index):
    sentence = train["text"][index][:-1] #отбросим точку в конце
    tagging = []
    iob_tags = parse_iob_tags(index)
    for word in sentence.split():
        tagging.append(iob_tags[word])
    return tagging

In [309]:
gapping_indexes = list(train[train["class"] == 1].index.values)

In [319]:
train = train.fillna("0:0")

In [323]:
tag_iob = []
for i in range(1, train.shape[0] + 1):
    if i in gapping_indexes:
        tag_iob.append(iob_tagging(i))
    else:
        tag_iob.append(None)

In [324]:
train["iob_tags"] = tag_iob

#### 4. Конвертер разметки-2 (1 балл)
Напишите функцию, переводящую разметку по оффсетам в скобочную запись. Она пригодится вам при анализе получаемых результатов.

In [370]:
def bracket_tagging(index):
    words = defaultdict()
    tuple_tag = defaultdict()
    sent = train["text"][index]
    tags = ["cV", "cR1", "cR2", "V", "R1", "R2"]
    # делаем словарь со слайсами и тэгами
    for tag in tags:
        for element in train[tag][index].split():
            nums = element.split(":")
            tuple_tag[(int(nums[0]), int(nums[1]))] = tag
    # сортируем тюплы в обратном порядке, чтобы не ломалась разметка
    sorted_by_first = sorted(tuple_tag.keys(), key=lambda tup: tup[0], reverse=True)
    # размечаем с конца, чтобы уже известные тюплы не уезжали при каждом шаге
    for element in sorted_by_first:
        # с тегом V все сложно, ломал все остальное. поэтому ставится решетка, которая заменяется в самом конце
        if tuple_tag[element] == "V":
            sent = sent[:element[0]-1] + "#" + sent[element[0]:]
        else:
            sent = sent[:element[0]] + tuple_tag[element] + "[ " + sent[element[0]:element[1]] \
            + " " + tuple_tag[element] + "]" + sent[element[1]:]
    sent = re.sub("#", " V[] ", sent)

    return sent

In [372]:
print(bracket_tagging(1))

Работа с двухбайтовыми наборами символов — просто кошмар для программиста, так как cR1[ часть их cR1] cV[ состоит cV] из cR2[ одного cR2] байта, а R1[ часть R1] — V[] R2[ из двух R2].


#### 5. Бинарная классификация (6 баллов (+2))
Постройте простую полносвязную сеть, берущую на вход все предложение и на выходе предсказывающую наличие или отсутствие гэппинга в предложении. Признаки: использование эмбеддингов +1 балл, использование pos-тэгов +1 балл

In [374]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from string import punctuation
punct = punctuation+'«»—…“”*№–'

In [446]:
pos_dict = {
    "NOUN": 1,
    "ADJF": 2,
    "ADJS": 3,
    "COMP": 4,
    "VERB": 5,
    "INFN": 6,
    "PRTF": 7,
    "PRTS": 8,
    "GRND": 9,
    "NUMR": 10,
    "ADVB": 11,
    "NPRO": 12,
    "PRED": 13,
    "PREP": 14,
    "CONJ": 15,
    "PRCL": 16,
    "INTJ": 17,
    None: 18}

In [464]:
#функция для создания векторов признаков из частеречной разметки
def pos_tagging(index):
    sentence = train["text"][index]
    pos_tags = []
    words = [word.strip(punct) for word in sentence.lower().split()]
    for word in words:
        pos_tags.append(pos_dict[morph.parse(word)[0].tag.POS])
    for i in range(len(pos_tags) - 1, 99):
        pos_tags.append(0)
    return pos_tags

In [390]:
import numpy as np

In [465]:
Xtrain = np.array([])

In [466]:
for index in range(train.shape[0]):
    Xtrain = np.append(Xtrain, pos_tagging(index))

In [475]:
# откуда-то берутся лишние 500 признаков, так и не узнала, откуда, поэтому тут костыль
Xtrain = Xtrain[:1640600].reshape(16406, 100)

In [418]:
ytrain = train["class"].to_list()

In [419]:
from sklearn.linear_model import Perceptron

In [479]:
clf = Perceptron(random_state=42)
clf.fit(Xtrain, ytrain)

/Users/isupovand/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
      fit_intercept=True, max_iter=None, n_iter=None, n_iter_no_change=5,
      n_jobs=None, penalty=None, random_state=42, shuffle=True, tol=None,
      validation_fraction=0.1, verbose=0, warm_start=False)

#### 6. Оценка результатов бинарной классификации (1 балл)
Обученной сеткой разметьте тестовые данные, с помощью готового скрипта agrr_metrics.py оцените полученный результат.
Пример вызова из командной строки:
> python3 agrr_metrics.py -b test.csv output.csv


In [481]:
def pos_tagging(index):
    sentence = test["text"][index]
    pos_tags = []
    words = [word.strip(punct) for word in sentence.lower().split()]
    for word in words:
        pos_tags.append(pos_dict[morph.parse(word)[0].tag.POS])
    for i in range(len(pos_tags) - 1, 99):
        pos_tags.append(0)
    return pos_tags

In [483]:
Xtest = np.array([])
for index in range(test.shape[0]):
    Xtest = np.append(Xtest, pos_tagging(index))

In [487]:
#тут тоже костыль, 3 лишний признака появляются из ниоткуда
Xtest = Xtest[:204500].reshape(2045, 100)

In [490]:
predicted = clf.predict(Xtest)

In [525]:
#работа с файлами не взлетела, поэтому скопировала сюда
def binary_metrics(y_true, y_pred):
    true_pos,false_pos,false_neg = 0,0,0
    for i in range(len(y_true)):
        if y_true[i] == 1 == y_pred[i]:
            true_pos += 1
        elif y_true[i] == 1 and y_pred[i] == 0:
            false_neg += 1
        elif y_true[i] == 0 and y_pred[i] == 1:
            false_pos += 1
    precision = true_pos/(true_pos + false_pos)
    recall = true_pos/(true_pos + false_neg)
    f1_score = 2*(precision*recall)/(precision + recall)
    return {'f1-score':f1_score, 'precision':precision,
             'recall':recall}

In [527]:
binary_metrics(list(test["class"].to_list()), predicted)

{'f1-score': 0.31936758893280637,
 'precision': 0.3452991452991453,
 'recall': 0.29705882352941176}

In [528]:
#это чуть лучше, чем случайный классификатор
from sklearn.metrics import roc_auc_score
roc_auc_score(list(test["class"].to_list()), predicted)

0.5082363714716656

<img src="https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/09/0995974824f37d2dcc303f5d3a0b22e1d59508e2_full.jpg">

#### 7. Классификация последовательности (7 баллов) 
Решите задачу полной разметки как классификацию последовательности. Напишите RNN, которая берет на вход последовательность символов длины n, выдает n IOB/IO/BIOS меток. [реализация NER от Stanford](https://cs230-stanford.github.io/pytorch-nlp.html)

#### 8. Оценка результатов полной разметки (1 балл)
Обученной сеткой разметьте тестовые данные, с помощью готового скрипта agrr_metrics.py оцените полученный результат.
Пример вызова из командной строки:
> python3 agrr_metrics.py test.csv output.csv

#### 9. Анализ результатов (до 4 баллов)
Проведите анализ ошибок. Посмотрите на fn, fp примеры, приведите несколько таких предложений в скобочной записи (если вы решали задачу классификации последовательности). Можно ли попытаться обобщить получаемые ошибки?